In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
import os 
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
class ByteEmbedding(nn.Module):
    def __init__(self,d_model,hash_size):
        super().__init__()
        self.byte_embed = nn.Embedding(256,d_model)
        self.hash_embed = nn.Embedding(hash_size,d_model)

    def forward(self,byte_seq,hash_seq):
        byte_embedding = self.byte_embed(byte_seq)
        hash_embedding = self.hash_embed(hash_seq)
        return byte_embedding + hash_embedding

In [3]:
class FeedForwardLayer(nn.Module):
    def __init__(self,d_model,ff_dim,dropout):
        super().__init__()
        self.layer1 = nn.Linear(d_model,ff_dim)
        self.layer2 = nn.Linear(ff_dim,d_model)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        return self.layer2(self.dropout(self.gelu(self.layer1(x))))

In [4]:
class TransformerBlock(nn.Module):
    def __init__(self,d_model,n_heads,ff_dim,dropout):
        super().__init__()
        self.attention = nn.MultiheadAttention(d_model,n_heads,dropout=dropout)
        self.ff = FeedForwardLayer(d_model,ff_dim,dropout=dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x,attn_mask = None):
        attn_out,_ = self.attention(x,x,x,attn_mask=attn_mask) 
        x = x + self.dropout(attn_out)
        x = self.norm1(x)
        ff_out = self.ff(x)
        x = x + self.dropout(attn_out)
        return self.norm2(x)

In [5]:
class CrossAttentionBlock(nn.Module):
    def __init__(self,query_dim,key_dim,n_heads,ff_dim,dropout):
        super().__init__()
        self.attention = nn.MultiheadAttention(query_dim,n_heads,dropout=dropout)
        self.norm = nn.LayerNorm(query_dim)
        self.ff = FeedForwardLayer(query_dim,ff_dim,dropout = dropout)
        self.dropout = nn.Dropout(dropout)
        self.query_proj = nn.Linear(query_dim,query_dim)
        self.key_proj = nn.Linear(key_dim,query_dim)
        self.value_proj = nn.Linear(key_dim,query_dim)

    def forward(self,query,key,value):
        query = self.query_proj(query).permute(1,0,2)
        key = self.key_proj(key).permute(1,0,2)
        value = self.value_proj(value).permute(1,0,2)

        attn_out , _ = self.attention(query,key,value)
        attn_out = attn_out.permute(1,0,2)
        query = query.permute(1,0,2)
        query = query + self.dropout(attn_out)
        query = self.norm(query)
        ff_out = self.ff(query)
        return query + self.dropout(ff_out)

In [6]:
class LocalEncoder(nn.Module):
    def __init__(self,byte_dim,n_heads,ff_dim,n_layers,dropout):
        super().__init__()
        self.layers = nn.ModuleList([TransformerBlock(byte_dim,n_heads,ff_dim,dropout) for _ in range(n_layers)])
        self.cross_attn = CrossAttentionBlock(query_dim=byte_dim,key_dim=byte_dim,n_heads=n_heads,ff_dim=ff_dim,dropout=dropout)

    def forward(self,byte_embeddings,patch_embeddings):
        for layer in self.layers:
            byte_embeddings = layer(byte_embeddings)
        patch_embedding = self.cross_attn(patch_embeddings,byte_embeddings,byte_embeddings)
        return patch_embedding
        

In [7]:
class LatentGlobalTransformer(nn.Module):
    def __init__(self,patch_dim,n_heads,ff_dim,n_layers,dropout):
        super().__init__()
        self.layers = nn.ModuleList([TransformerBlock(patch_dim,n_heads,ff_dim,dropout) for _ in range(n_layers)])
    def forward(self,patches,attn_mask=None):
        for layer in self.layers:
            patches = layer(patches,attn_mask=attn_mask)
        return patches

In [8]:
class LocalDecoder(nn.Module):
    def __init__(self,patch_dim,byte_dim,n_heads,ff_dim,n_layers,dropout):
        super().__init__()
        self.layers = nn.ModuleList([TransformerBlock(byte_dim,n_heads,ff_dim,dropout) for _ in range(n_layers)])
        self.cross_attn = CrossAttentionBlock(query_dim=byte_dim,key_dim=patch_dim,n_heads=n_heads,ff_dim=ff_dim,dropout=dropout)
        self.output_proj = nn.Linear(byte_dim,256)

    def forward(self,patch_embedding,byte_embedding):
        byte_embedding = self.cross_attn(byte_embedding,patch_embedding,patch_embedding)
        for layer in self.layers:
            byte_embedding = layer(byte_embedding)
        return self.output_proj(byte_embedding)

In [9]:
class ByteLatentTransformer(nn.Module):
    def __init__(self,byte_dim,patch_dim,vocab_size,n_heads,ff_dim,n_encoder,n_decoder,n_global,dropout=0.1):
        super().__init__()
        self.byte_embed = ByteEmbedding(byte_dim,vocab_size)
        self.local_encoder = LocalEncoder(byte_dim,n_heads,ff_dim,n_layers=n_encoder,dropout=dropout)
        self.global_transformer = LatentGlobalTransformer(patch_dim,n_heads,ff_dim,n_layers=n_global,dropout=dropout)
        self.local_decoder = LocalDecoder(patch_dim,byte_dim,n_heads,ff_dim,n_decoder,dropout=dropout) 
        self.projection = nn.Linear(byte_dim,patch_dim)

    def forward(self,byte_seq,hash_seq,patch_seq):
        byte_embeddings = self.byte_embed(byte_seq,hash_seq)
        if patch_seq is None:
            patch_embeddings = torch.mean(byte_embeddings,dim=1,keepdim=True)
            patch_embeddings = self.local_encoder(byte_embeddings,patch_embeddings)
            patch_embeddings = self.projection(patch_embeddings)
        else:
            patch_embeddings = patch_seq
        patch_embeddings = self.global_transformer(patch_embeddings)
        byte_output = self.local_decoder(patch_embeddings,byte_embeddings)
        return byte_output

In [10]:
print(torch.cuda.is_available())

True


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import os
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt

# -----------------------
# 1. Load and Prepare Data
# -----------------------
wiki = load_dataset("wikitext", "wikitext-2-raw-v1")
# Use a small subset for quicker training; adjust these as needed.
train_size = int(len(wiki["train"]) * 0.1)
val_size = int(len(wiki["validation"]) * 0.1)

# Shuffle and select only the subset
train_subset = wiki["train"].shuffle(seed=42).select(range(train_size))
val_subset = wiki["validation"].shuffle(seed=42).select(range(val_size))

# Use the subset texts instead of the full dataset
train_text = "\n".join(train_subset["text"])
val_text = "\n".join(val_subset["text"])

# Convert text to a tensor of raw byte values using UTF-8 encoding
train_bytes = torch.tensor(list(train_text.encode('utf-8')), dtype=torch.long)
val_bytes = torch.tensor(list(val_text.encode('utf-8')), dtype=torch.long)

# -----------------------
# 2. Create a Dataset Class with Deterministic Hashing
# -----------------------
class WikiTextDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def deterministic_hash(self, input_seq):
        # A simple deterministic hash: multiply by 31, add 17, modulo 256.
        return (input_seq * 31 + 17) % 256

    def __getitem__(self, idx):
        input_seq = self.data[idx:idx + self.seq_len]
        target_seq = self.data[idx + 1:idx + self.seq_len + 1]
        # Compute hash deterministically from the input sequence
        hash_seq = self.deterministic_hash(input_seq)
        return input_seq, hash_seq, target_seq

seq_len = 128
batch_size = 64

train_dataset = WikiTextDataset(train_bytes, seq_len)
val_dataset = WikiTextDataset(val_bytes, seq_len)

# Increase number of workers and enable pin_memory for faster data loading (if using GPU)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

byte_dim = 128
patch_dim = 256
vocab_size = 256  # Bytes: 0-255
n_heads = 8
ff_dim = 1024
n_encoder = 4
n_decoder = 4
n_global = 6
dropout = 0.1
epochs = 100
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ByteLatentTransformer(
    byte_dim=byte_dim,
    patch_dim=patch_dim,
    vocab_size=vocab_size,
    n_heads=n_heads,
    ff_dim=ff_dim,
    n_encoder=n_encoder,
    n_decoder=n_decoder,
    n_global=n_global,
    dropout=dropout
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# -----------------------
# 4. Checkpoint Utilities
# -----------------------
def save_checkpoint(epoch, model, optimizer, train_loss, val_loss, best_val_loss, filepath="checkpoint.pth"):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'best_val_loss': best_val_loss
    }
    torch.save(checkpoint, filepath)
    print(f"Checkpoint saved at epoch {epoch}!")

def load_checkpoint(filepath="checkpoint.pth"):
    if os.path.exists(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        print(f"Checkpoint loaded from epoch {checkpoint['epoch']}!")
        return checkpoint
    else:
        print("No checkpoint found!")
        return None

# -----------------------
# 5. Helper Functions: Plot Loss and Generate Text with Temperature Sampling
# -----------------------
def plot_loss(train_losses, val_losses):
    plt.figure(figsize=(10,6))
    plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss', marker='o')
    plt.plot(range(1, len(val_losses)+1), val_losses, label='Val Loss', marker='o')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.savefig('loss_plot.png')
    plt.show()

def generate_text_for_sample(model, input_seq, device, length, temperature=1.0):
    model.eval()
    # Compute deterministic hash for the input sequence
    hash_seq = (input_seq * 31 + 17) % 256
    generated_text = "".join([chr(x.item()) for x in input_seq[0]])
    for _ in range(length):
        with torch.no_grad():
            output = model(input_seq, hash_seq, None)
            logits = output[0, -1] / temperature
            probs = torch.softmax(logits, dim=-1)
            next_byte = torch.multinomial(probs, num_samples=1).item()
            generated_text += chr(next_byte)
            input_seq = torch.cat([input_seq[:, 1:], torch.tensor([[next_byte]], device=device)], dim=1)
            hash_seq = (input_seq * 31 + 17) % 256
    return generated_text

# -----------------------
# 6. Training Loop with Mixed Precision, Early Stopping, Checkpointing, and Logging
# -----------------------
train_losses = []
val_losses = []
early_stop_patience = 5
epochs_no_improve = 0

def train_model():
    global train_losses, val_losses, epochs_no_improve
    best_val_loss = float('inf')
    start_epoch = 0
    checkpoint = load_checkpoint()
    if checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        train_losses = checkpoint['train_loss']
        val_losses = checkpoint['val_loss']
        best_val_loss = checkpoint['best_val_loss']
        start_epoch = checkpoint['epoch']
    
    scaler = torch.amp.GradScaler()
    
    for epoch in range(start_epoch, epochs):
        model.train()
        tot_loss = 0
        print(f"Epoch [{epoch+1}/{epochs}]")
        for i, (byte_seq, hash_seq, target_seq) in enumerate(tqdm(train_loader, desc="Training")):
            byte_seq = byte_seq.to(device, non_blocking=True)
            hash_seq = hash_seq.to(device, non_blocking=True)
            target_seq = target_seq.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            with torch.amp.autocast(device_type="cuda"):
                outputs = model(byte_seq, hash_seq, None)
                if i == 0:
                    print("Logits stats: mean: {:.4f}, std: {:.4f}, min: {:.4f}, max: {:.4f}".format(
                        outputs.mean().item(), outputs.std().item(), outputs.min().item(), outputs.max().item()))
                outputs = outputs.view(-1, vocab_size)
                target_seq = target_seq.view(-1)
                loss = criterion(outputs, target_seq)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            tot_loss += loss.item()
        
        avg_train_loss = tot_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        model.eval()
        tot_val_loss = 0
        with torch.no_grad():
            for byte_seq, hash_seq, target_seq in tqdm(val_loader, desc="Validation"):
                byte_seq = byte_seq.to(device, non_blocking=True)
                hash_seq = hash_seq.to(device, non_blocking=True)
                target_seq = target_seq.to(device, non_blocking=True)
                with torch.amp.autocast(device_type="cuda"):
                    outputs = model(byte_seq, hash_seq, None)
                    outputs = outputs.view(-1, vocab_size)
                    target_seq = target_seq.view(-1)
                    l = criterion(outputs, target_seq)
                tot_val_loss += l.item()
        avg_val_loss = tot_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), "BLT_wikitext.pth")
            print("Best model saved!")
        else:
            epochs_no_improve += 1
            print(f"No improvement in validation loss for {epochs_no_improve} epochs.")
            if epochs_no_improve >= early_stop_patience:
                print("Early stopping triggered.")
                break

        save_checkpoint(epoch+1, model, optimizer, train_losses, val_losses, best_val_loss)
        print("Sample Generated Text:")
        sample_prompt = "The "  # A common prompt in Wiki text
        sample_input = torch.tensor([ord(c) for c in sample_prompt], dtype=torch.long).unsqueeze(0).to(device)
        print(generate_text_for_sample(model, sample_input, device, length=seq_len, temperature=0.8))
    
    plot_loss(train_losses, val_losses)

# Start training
train_model()


/tmp/ipykernel_6481/1815865120.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath, map_location=device)


Checkpoint loaded from epoch 40!
Epoch [41/100]


Training:   0%|                               | 1/17428 [00:00<31:45,  9.15it/s]

Logits stats: mean: -9.6875, std: 6.9922, min: -25.2188, max: 13.4062


Validation: 100%|███████████████████████████| 1843/1843 [00:20<00:00, 89.65it/s]


...

Validation: 100%|███████████████████████████| 1843/1843 [00:20<00:00, 89.83it/s]


Epoch [60/100], Train Loss: 2.1835, Validation Loss: 2.2300
No improvement in validation loss for 3 epochs.
Checkpoint saved at epoch 60!
Sample Generated Text:
The The Thelin ien iondiong angannd III IIIII II I I I I w I I II II I I I I I Ithata al all annnnennntrntrrerroroly onthen . ft ati
Epoch [61/100]


Training:   0%|                               | 1/17428 [00:00<31:02,  9.36it/s]

Logits stats: mean: -11.6875, std: 8.5469, min: -34.7188, max: 18.0938


Training:  69%|██████████████████▌        | 11981/17428 [07:08<03:13, 28.09it/s]